# Costi Riscossione Tributi

Questa analisi ha come obiettivo approfondire come i costi per **la riscossione dei tributi** si distribuisce in Italia. In particolare:
- Analizzeremo la sua distribuzione normalizzata per il numero di contribuenti per le varie Regioni.
- Cercheremo di capire quali sono le top/last regioni/province/comuni per spesa riscossione tributi

In [1]:
import numpy as np
import pandas as pd

# set the max number of columns to display
pd.options.display.max_columns = 98

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

import plotly.offline as py
import plotly.plotly as py_on
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)

from IPython.display import Image
from IPython.display import display

Il dataset preso in analisi contiene informazioni sui **costi riscossione tributi per 2010-2013 (OpenCivitas)**  e i **redditi dei contribuenti (Mef dati Irpef)** per lo stesso periodo. In totale abbiamo 26800 osservazioni distribuite per anno:

In [5]:
fattore = 'costo riscossione tributi'
fattore_descrizione = 'Costo Riscossione Tributi'

tot_contribuenti = 'tot_contribuenti'

fattore_normalizzato = 'spesa_tributi_per_contribuenti'

In [6]:
df = pd.read_csv('./data/opencivitas_spesa_storica_dati_irpef.csv')
df.shape

(26799, 86)

Distribuzione delle osservazioni Anno e Comuni per cui abbiamo rilevazioni per l'anno in questione.

In [7]:
df.groupby(['ANNO'])[fattore].count()

ANNO
2010    6702
2011    6699
2012    6699
2013    6699
Name: costo riscossione tributi, dtype: int64

Aggiungiamo ora la colonna per il fattore di analisi normalizzato per numero di contribuenti

In [8]:
df[fattore_normalizzato]= df[fattore] / df[tot_contribuenti]
df[fattore_normalizzato].head()

0     9.135789
1     0.233289
2    20.696801
3     6.788800
4    38.156980
Name: spesa_tributi_per_contribuenti, dtype: float64

-----------------------

In [ ]:
df_regione = df.groupby(df.Regione)[fattore_normalizzato].mean()
df_regione = df_regione.sort_values(ascending=False)

In [ ]:
data_pie = [
    go.Pie(labels = df_regione.index, values=df_regione),
]

layout_pie = go.Layout(
    title='Costo medio riscossione tasse per Regione/Contribuente in Euro'
)

fig_pie = go.Figure(data=data_pie, layout=layout_pie)

data_bar = [
    go.Bar(x=df_regione.index, y=df_regione, name = 'spesa media')
]

layout_bar = go.Layout(
    title='Costo medio riscossione tasse per Regione/Contribuente in Euro',
    xaxis=dict(
        title='Regioni',
    ),
    yaxis=dict(
        title='Euro per Contribuente',
    )
)

fig_bar = go.Figure(data=data_bar, layout=layout_bar)

## Costo Riscossione Tributi per Regione

Per prima cosa analizziamo quali sono le regioni che hanno maggiori costi per la riscossione delle tasse.
Possiamo osservare che per il periodo 2010-2013 la Liguria sia la regione con li maggior costo procapite per la riscossione dei tributi. La tabella sotto mostra il dettaglio dei costi. 

Mentre, il grafico a torta fa vedere la distribuzione percentuale di questi costi sul totale delle regioni presenti nel dataset.

In [ ]:
df_regione

In [ ]:
py.iplot(fig_pie)
py.iplot(fig_bar)

fig_pie_path = 'imgs/tributi_regione_pie.png'
fig_bar_path = 'imgs/tributi_regione_bar.png'

py_on.image.save_as(fig_pie, filename=fig_pie_path)
py_on.image.save_as(fig_bar, filename=fig_bar_path)

x = Image(fig_pie_path)
y = Image(fig_bar_path)
display(x,y)

## Dettaglio Riscossione Tributi Puglia, Campania e Lombardia 

E' interessante notare come Puglia, Campania e Lombardia abbiano un costo di riscossione molto simile. Andiamo a dettagliare questi dati rispetto al totale costi e al numero di persone per avere una idea. 
Inoltre, possiamo incrociare questi dati con i dati Irpef per capire quali sono le fasce di reddito che contribuiscono maggiormente agli introiti di cassa delle regioni.


In [ ]:
# mediamo sul numero degli anni in analisi
df_regione_sum = df.groupby(df.Regione).sum() / 4

fattori_analisi = [fattore, tot_contribuenti]
regioni_interesse = ['PUGLIA','LOMBARDIA','CAMPANIA']

df_tributi_contribuenti = df_regione_sum[fattori_analisi]

df_regioni_interesse = df_tributi_contribuenti.loc[regioni_interesse,:]
df_regioni_interesse = df_regioni_interesse.sort_values(tot_contribuenti, ascending=False)

In [ ]:
data = [
    go.Bar(x=df_regioni_interesse.index, y=df_regioni_interesse[fattore], name='costo riscossione tributi'),
    go.Bar(x=df_regioni_interesse.index, y=df_regioni_interesse[tot_contribuenti], name='numero contribuenti'),  
]

layout = go.Layout(
    title='Costo totale riscossione tributi e numero contribuenti',
    xaxis=dict(
        title='Regioni',
    ),
    yaxis=dict(
        title='Totale Costi - numero contribuenti',
    )
)
fig = go.Figure(data=data, layout=layout)

In [ ]:
py.iplot(fig)

fig_path = 'imgs/totale_riscossione_tributi.png'
py_on.image.save_as(fig, filename=fig_path)
x = Image(fig_path)
display(x)

>Andiamo ad analizzare in dettaglio il numero dei contribuenti nelle Regioni considerate.
Se confrontiamo questo valore con la popolazione residente, possiamo avere una idea della percentuale della popolazione che dichiara introiti.

|   |   |   |   |
|---|---|---|---|
| LOMBARDIA  |  7026749  | 10000000 | 70.2% |
| CAMPANIA   |  3085606  |  5869000 | 52.5% |
| PUGLIA     |  2541365  |  4087000 | 62.1% |


### Distribuzione Redditi

Analizziamo ora come sia distribuita la contribuzione fiscale da parte dei contribuenti.
In particolare, analizziamo le diverse fonti di reddito dal punto di vista di:

1. distribuzione contribuenti
2. distribuzione introiti

Qui sotto consideriamo quali sono i fattori per l'analisis:

In [ ]:
fattori = [c for c in df.columns.values if c.startswith('Reddito') and 'Frequenza' in c]
fattori

Dai quali selezioniamo i redditi da lavoro

In [ ]:
frequenza_reddito_scaglioni = [
 ('Regione','Reddito minore o uguale a zero Frequenza', '0M'),
 ('Regione','Reddito complessivo da 0 a 10000 euro Frequenza', '0-10M'),
 ('Regione','Reddito complessivo da 10000 a 15000 euro Frequenza', '10-15M'),
 ('Regione','Reddito complessivo da 15000 a 26000 euro Frequenza', '15-26M'),
 ('Regione','Reddito complessivo da 26000 a 55000 euro Frequenza', '26-55M'),
 ('Regione','Reddito complessivo da 55000 a 75000 euro Frequenza', '55-75M'),
 ('Regione','Reddito complessivo da 75000 a 120000 euro Frequenza', '75-120M'),
 ('Regione','Reddito complessivo oltre 120000 euro Frequenza', '>120M'),
]

fattori_frequenza = [v for _, v, _ in frequenza_reddito_scaglioni]

### Frequenza contribuenti per fascia economica

L'istogramma e la tabella sotto mostra la frequenza di contribuenti per fascia di reddito.
Inoltre, di seguito viene mostrata la stessa quantita' in valore percentuale. Questo serve a vedere la contribuzione delle varie fasce di reddito per Regione di interesse.

In [ ]:
df_regioni_redditi

### Contribuzione per fascia di reddito in percentuale

In [ ]:
frequenza_redditi_percentuale = df_regione_sum.loc[regioni_interesse,fattori_frequenza].reset_index()
tot_contribuenti_regione = df_regione_sum.loc[regioni_interesse,tot_contribuenti].reset_index()

frequenza_redditi_percentuale[fattori_frequenza] = frequenza_redditi_percentuale[fattori_frequenza].div(
    tot_contribuenti_regione[tot_contribuenti], axis='index')
frequenza_redditi_percentuale

In [ ]:
data_frequenza = []
for x,y,name in fattori_reddito_scaglioni:
    bar = go.Bar(
        x = frequenza_redditi_percentuale[x],
        y = frequenza_redditi_percentuale[y],
        name = name
    )
    data_frequenza.append(bar)

layout = go.Layout(
    title='Distribuzione per Fasce di Contribuzione - Frequenza',
    xaxis=dict(
        title='Regioni',
    ),
    yaxis=dict(
        title='Percentuale',
    )
)

fig = go.Figure(layout=layout, data=data_frequenza)
py.iplot(fig)

fig_path = 'imgs/percentuale_contribuenti_reddito.png'
py_on.image.save_as(fig, filename=fig_path)
x = Image(fig_path)
display(x)

Da questo grafico si puo' osservare come la Lombardia abbian una distribuzione spostata verso le alte fasce di contribuzione. Di seguito confrontiamo questo grafico con uno relativo alla contribuzione in termini di **Ammontare** di} denaro.

In [ ]:
ammontare_reddito_scaglioni = [
 ('Regione','Reddito minore o uguale a zero Ammontare', '0M'),
 ('Regione','Reddito complessivo da 0 a 10000 euro Ammontare', '0-10M'),
 ('Regione','Reddito complessivo da 10000 a 15000 euro Ammontare', '10-15M'),
 ('Regione','Reddito complessivo da 15000 a 26000 euro Ammontare', '15-26M'),
 ('Regione','Reddito complessivo da 26000 a 55000 euro Ammontare', '26-55M'),
 ('Regione','Reddito complessivo da 55000 a 75000 euro Ammontare', '55-75M'),
 ('Regione','Reddito complessivo da 75000 a 120000 euro Ammontare', '75-120M'),
 ('Regione','Reddito complessivo oltre 120000 euro Ammontare', '>120M'),
]

ammontare_fattori = [v for _, v, _ in ammontare_reddito_scaglioni]

In [ ]:
ammontare_redditi_percentuale = df_regione_sum.loc[regioni_interesse,ammontare_fattori].reset_index()
redditi_tot = df_regione_sum.loc[regioni_interesse, ammontare_fattori].sum(axis=1).reset_index()

ammontare_redditi_percentuale[ammontare_fattori]= redditi_interesse[ammontare_fattori].div(
    redditi_tot[0], axis='index'
)
ammontare_redditi_percentuale

In [ ]:
data_ammontare = []
for x,y,name in ammontare_reddito_scaglioni:
    bar = go.Bar(
        x = ammontare_redditi_percentuale[x],
        y = ammontare_redditi_percentuale[y],
        name = name
    )
    data_ammontare.append(bar)

layout = go.Layout(
    title='Distribuzione per Fasce di Contribuzione - Ammontare',
    xaxis=dict(
        title='Regioni',
    ),
    yaxis=dict(
        title='Percentuale',
    ),
    barmode='stack'
)

fig = go.Figure(layout=layout, data=data_ammontare)
py.iplot(fig)

fig_path = 'imgs/percentuale_contribuenti_reddito_ammontare.png'
py_on.image.save_as(fig, filename=fig_path)
x = Image(fig_path)
display(x)


# figs = tools.make_subplots(rows=1, cols=2, shared_xaxes=True, shared_yaxes=True)
# for d in data_frequenza:
#     figs.append_trace(d,1,1)
# for d in data_ammontare:
#     figs.append_trace(d,1,2)   

# py.iplot(figs)

In [ ]:
ammontare_redditi_percentuale


Per avere una visione comparabile delle voci di reddito possiamo analizzarle anche sotto forma di **line chart**

#### Line Chart


In [ ]:
trasposta = ammontare_redditi_percentuale.T
trasposta.columns = trasposta.iloc[0]
trasposta = trasposta.iloc[1:,:]

columns_dict = {d:v for _, d, v in ammontare_reddito_scaglioni}
line_x = [columns_dict[k] for k in trasposta.index]
line_x

line_data = []
for c in trasposta:
    data = trasposta[c].values
    line_data.append(
        go.Scatter(
            x=line_x,
            y=data, 
            name=c,     
            fill='tozeroy',
            mode= 'none'
        )
    )

layout = go.Layout(
    title='Distribuzione per Fasce di Contribuzione - Ammontare',
)
    
fig = go.Figure(layout=layout, data=line_data)
py.iplot(fig)

fig_path = 'imgs/percentuale_contribuenti_reddito_line.png'
py_on.image.save_as(fig, filename=fig_path)
x = Image(fig_path)
display(x)


Da qui possiamo vedere ancora meglio come la Lombardia abbia una distribuzione dei valori diversa da quelle di Puglia e Campania.

## Confronto Liguria e Veneto

A questo punto sembra interessante controntare la Liguria e il Veneto per capire da cosa possa derivare una cosi' grande discrepanza.
Prendiamo in considerazione i costi sostenuti dalle due regioni per i vari servizi.

In [ ]:
df_regione_tot = df.groupby(df.Regione).sum() / 4

regioni_confronto = ['LIGURIA','VENETO']
voci_di_costo = [v for v in df_regione_tot.columns if v.startswith('SPESA')]
voci_di_cost_desc = [
    'riscossione tributi',
    'ufficio tecnico',
    'anagrafe, stato civile',
    'altri servizi',
    'polizia locale',
    'istruzione',
    "viabilita'",
    "trasporti",
    "territorio",
    "ambiente",
    "sociale",
    "asili nido",
]

In [ ]:
df_regione_spesa_perc = df_regione_tot[voci_di_costo].div(df_regione_tot['SPESA_CCC_tot'], axis='index')

In [ ]:
df_regione_spesa_perc = df_regione_spesa_perc.drop('SPESA_CCC_tot', axis=1)
df_regione_spesa_perc.columns = voci_di_cost_desc

In [ ]:
df_regione_spesa_perc_an = df_regione_spesa_perc.loc[regioni_confronto]
df_regione_spesa_perc_an = df_regione_spesa_perc_an.reset_index()
df_regione_spesa_perc_an

In [ ]:
data = []
for c in df_regione_spesa_perc_an.columns.values[1:]:
    bar = go.Bar(
        x = regioni_confronto,
        y = df_regione_spesa_perc_an[c],
        name = c
    )
    data.append(bar)

layout = go.Layout(
    title='Percentuale Costi per Tipologia di Servizio',
    xaxis=dict(
        title='Regioni',
    ),
    yaxis=dict(
        title='Percentuale',
    )
)

fig = go.Figure(layout=layout, data=data)
py.iplot(fig)

fig_path = 'imgs/costo_per_servizio.png'
py_on.image.save_as(fig, filename=fig_path)
x = Image(fig_path)
display(x)

In [ ]:
df_regione_spesa_perc_reset = df_regione_spesa_perc.reset_index()

In [ ]:
fig = ff.create_scatterplotmatrix(
    df_regione_spesa_perc_reset.iloc[:,0:5], 
    index='Regione',
    height=800, width=800)
py.iplot(fig)

### Q-Q Plot

In [ ]:
column = df_regione_spesa_perc['riscossione tributi']
data = []
for k,v in column.iteritems():
    data.append(
        go.Scatter(
            x = [v],
            y = [v],
            name = k
        )
    )
    
layout = go.Layout(
    title='Percentuale Costi per Riscossione Tributi',
)
    
fig = go.Figure(layout=layout, data=data)
py.iplot(fig)